In [1]:
import os
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set up the model
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-1106-preview")
parser = StrOutputParser()

In [4]:
prompt = ChatPromptTemplate.from_template(template)

In [5]:
pregunta="¿Quién fue el primer ingeniero egresado de la UBA?"

## Primero probamos hacer una pregunta directa

In [6]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": ""},
    {"role": "user", "content": pregunta}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='No tengo información específica sobre quién fue el primer ingeniero egresado de la Universidad de Buenos Aires (UBA) en Argentina. Sin embargo, puedo proporcionarte algo de contexto sobre la facultad responsable de esta área de estudio.\n\nLa Facultad de Ingeniería de la Universidad de Buenos Aires fue fundada en 1865, originalmente como parte del Departamento de Ciencias Exactas y luego evolucionó a la facultad actual. Si necesitas conocer quién fue el primer ingeniero egresado, podrías consultar los registros históricos de la UBA o ponerse en contacto directamente con la facultad.\n\nPuedo ayudarte con información más detallada o general sobre la historia de la facultad, programas de estudio o contribuciones significativas si estás interesado.', role='assistant', function_call=None, tool_calls=None)


In [3]:
#definimos un template
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "Yo que carajo sé".

Context: {context}

Question: {question}
"""

In [8]:
# Load the file containing laws
loader =loader = TextLoader("./textos/huergo.txt")
text_documents = loader.load()

# Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=25000, chunk_overlap=5000)
documents = text_splitter.split_documents(text_documents)

# Generate embeddings for the law chunks
embeddings = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [9]:
 retriever = vectorstore.as_retriever()
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())
chain = setup | prompt | model | parser

response = chain.invoke(pregunta)
print(response)

El primer ingeniero egresado de la Universidad de Buenos Aires (UBA) fue Luis Augusto Huergo.
